# 환경 설정

In [4]:
!pip install "langchain==0.2.6"
!pip install "ibm-watsonx-ai==1.0.10"
!pip install "langchain_ibm==0.1.8"
!pip install "langchain_community==0.2.6"
!pip install "sentence-transformers==3.0.1"
!pip install "chromadb==0.5.3"
!pip install "pydantic==2.8.2"
!pip install "langchain-huggingface==0.0.3"
!pip install "python-dotenv==1.0.1"
!pip install "pymupdf4llm==0.0.13"      # pdf markdown 형식으로 load

^C



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import re   # 정규표현식 활용
from dotenv import load_dotenv
import pymupdf4llm    # pdf markdown 형식으로 load
import pandas as pd   # 데이터 처리
from langchain_community.document_loaders import TextLoader   # txt 파일 load
from langchain_community.document_loaders.csv_loader import CSVLoader # csv 파일 load
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate     # ChatPrompt template
from langchain_core.prompts import PromptTemplate         # Rag-Prompt template 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnableParallel, RunnablePassthrough  # retriever data 병렬 전달
from langchain_core.output_parsers import StrOutputParser # chain output을 str로 받기 

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain.llms import WatsonxLLM

In [2]:
load_dotenv()

True

In [3]:
# IBM Cloud 상의 watsonx SAS url은 provision된 region별로 정해져 있음. https://cloud.ibm.com/apidocs/watsonx-ai 참조.
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": 'https://us-south.ml.cloud.ibm.com'
}

In [4]:
# embedding model, LLM model load
# embedding: huggingface의 snunlp/KR-SBERT embedding model 활용
embeddings = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",       # 한국어 특화 임베딩 모델
)

# LLM: watsonx.ai api를 활용하여 "mistralai/mistral-large" 모델 사용
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

model_id =  'mistralai/mistral-large' # 'meta-llama/llama-3-1-70b-instruct' # ModelTypes.LLAMA_2_70B_CHAT.value #
watsonx_mistralai = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    apikey=wml_credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

c:\Users\default.DESKTOP-T7K82N0\moneymentor\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2036\1028397337.py:16: LangChainDeprecationWarning: The class `WatsonxLLM` was deprecated in LangChain 0.0.18 and will be removed in 0.3. An updated version of the class exists in the langchain-ibm package and should be used instead. To use it run `pip install -U langchain-ibm` and import as `from langchain_ibm import WatsonxLLM`.
  watsonx_mistralai = WatsonxLLM(


In [5]:
# 기본 지식 정보(Knowledge base) 구축
# 1. 경제금융용어 데이터 처리
# 데이터 파일 경로 지정
pdf_filename = os.path.join(os.getcwd(), 'data', '한국은행_경제금융용어.pdf')

In [6]:
# pdf파일 markdown형태의 text로 불러오기 
financial_md_text = pymupdf4llm.to_markdown(pdf_filename)     # markdown 형식으로 pdf 읽기

# 불필요한 문자 제거
def preprocessing_text(text):  
    # 문서에 필요없는 앞 뒤 삭제
    start = re.search(r'\*\*', text).span()[0]  # 첫 번재 마크다운이 나오는 위치
    end = re.search(r'경제금융용어 700선 집필자', text).span()[0]
    text = text[start:end]

    # 불필요한 문자 제거
    text = text.replace("경제금융용어  700선", "")
    text = text.replace("\n\n연관검색어", " 연관검색어")
    text = re.sub(r'\n\n\n-----\n\n', ' ', text)
    text = text.replace("\n\n", " ")
    text = re.sub(r'[ㄱ-ㅎ]', '', text)  # 초성 제거

    # 나중에 chunk를 \n\n을 기준으로 할 수 있게 용어 앞 뒤 처리 
    text = text.replace("**", "\n\n")
    text = text.replace("\n\n ", " ")

    return text

# 파일에 문자열을 저장하는 함수
def save_to_txt_file(file_name, content):
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(content)

# 전처리 
clean_financial_md_text = preprocessing_text(financial_md_text)

# 문자열을 파일로 저장
save_to_txt_file('data/한국은행_경제금융용어.txt', clean_financial_md_text)

In [7]:
# 데이터 파일 경로 지정
txt_filename = os.path.join(os.getcwd(), 'data', '한국은행_경제금융용어.txt')

In [8]:
# txt 파일 불러오기
loader = TextLoader(txt_filename, encoding='utf-8')
financial_documents = loader.load()

# chunck "/n/n"을 기준으로
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0
)

financial_texts = text_splitter.split_documents(financial_documents)

In [9]:
# 2. 시사경제용어사전 데이터 처리
# xlsx 파일 csv 파일로 변환
xlsx_filename = os.path.join(os.getcwd(), 'data', '기획재정부_시사경제용어사전.xlsx')
data = pd.read_excel(xlsx_filename)
data.to_csv('data/기획재정부_시사경제용어사전.csv', index=False)
csv_filename = os.path.join(os.getcwd(), 'data', '기획재정부_시사경제용어사전.csv')
loader = CSVLoader(csv_filename, encoding='utf-8')
current_documents = loader.load()
# 데이터 전처리
def modify_page_content(doc):
    # 줄바꿈 문자를 공백으로 변경
    doc.page_content = doc.page_content.replace('\n', ' ')
    return doc

# 수정된 Document 객체 리스트 생성
current_texts = [modify_page_content(doc) for doc in current_documents]

In [10]:
# Vector DB 생성: ChromaDB 활용
# 경제금융용어 먼저 넣기
DB_PATH = "./chroma_db"

db = Chroma.from_documents(
    documents=financial_texts, embedding=embeddings, 
    persist_directory=DB_PATH, 
    collection_name="economic_terms_db"
)

In [11]:
# 시사경제용어 추가
db.add_documents(documents=current_texts)

['cbb41d9f-cc7b-4e25-884a-e27b2a8dadc2',
 '6d56b0e6-10e5-4862-bfd2-ad84f80d2a94',
 '5c5395b7-f2bd-4240-adf1-56b2447faf0f',
 '160facf8-a4e4-4409-9922-56c4e77aeb38',
 'c18cb353-dd23-49bf-8992-da4c81539bbc',
 '0cfd2dcd-e46c-4f0b-b775-873880129192',
 'a82f99e9-cf7e-41c4-8bdb-15668425ba1e',
 '316185be-5fe8-4179-861d-a9b689cae18f',
 '481fce91-20ad-4a20-b47c-ee8a93cb3519',
 '856958ed-7e2f-4707-a0a5-9e0b494424f0',
 'bfe2e51c-9e29-4b03-8495-d97d7e8f4b12',
 'c9a2f0da-cf7c-4cf0-9c98-024910664cce',
 'fd7ac06d-28de-4b46-85b8-edcc04101bc2',
 'b37270ea-fad0-4c93-921d-e20a78c8a429',
 'b816ab8a-ccaa-4647-9533-93d34f131810',
 '9a55b7bd-04ac-4711-be10-4d82e1413094',
 'b0da25c4-af92-4352-a292-65c01f01b926',
 'be501ec6-0a9b-45de-afa2-f1be95188210',
 'ed5c78e1-90f6-45d5-afad-17b7e0252ce1',
 'f774dab5-16e3-4d8b-a0c5-046eef5e6eb4',
 '656adefa-453d-45dd-8861-3a995df3c41c',
 '6928e06d-ea4a-4919-94c8-0ee54c273c24',
 'aedf7369-95a2-4f5e-90ae-831aa9ca7055',
 '2c4fe122-44c5-4645-8be5-1a7dac31895c',
 '759b80cd-7b70-

In [12]:
# DB 로드
db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="economic_terms_db",
)

In [13]:
# retriever 생성
retriever = db.as_retriever(search_kwargs={'k': 5})

In [14]:
# prompt 작성
prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# chain 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | watsonx_mistralai
    | StrOutputParser()
)

In [15]:
# 질의응답 테스트
user_input = "소득교역조건이 무엇인가요?"
rag_chain.invoke(user_input)

'\n소득교역조건은 순상품교역조건에 수출물량지수를 곱하여 {(수출단가지수/수입 단가지수)×수출물량지수}로 산출하는 것으로, 수출 금액으로 수입을 늘릴 수 있는 능력을 측정하는 지표입니다. 소득교역조건지수가 상승하였다면 수출총액으로 수입할 수 있는 능력(수량)이 좋아졌다는 것을 의미합니다.'

In [16]:
# retriever 테스트
retriever.get_relevant_documents(user_input)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_2036\3075586040.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(user_input)


[Document(metadata={'source': 'c:\\Users\\default.DESKTOP-T7K82N0\\moneymentor\\AI\\chatbot_economic_terms\\data\\한국은행_경제금융용어.txt'}, page_content='소득5분위배율 소득분배의 불평등도를 나타내는 지표의 하나로 전체 가구를 소득수준의 순서에 따라 20%씩 5등분으로 나눈 다음, 소득 상위 20%(5분위) 계층의 평균소득을 소득 하위 20%(1분위) 계층의 평균소득으로 나눈 값이다. 소득 5분위 배율 값이 커질수록 고소득층 과 저소득층 간의 소득분배가 악화하였음을 의미한다. 연관검색어 : 상대적 빈곤율, 지니계수 \n\n소득교역조건/소득교역조건지수 소득교역조건지수는 순상품교역조건에 수출물량지수를 곱하여 {(수출단가지수/수입 단가지수)×수출물량지수} 산출하는 것으로서 수출 금액으로 수입을 늘릴 수 있는 능력을 측정하는 지표이다. 소득교역조건지수가 상승하였다면 금기의 수출총액으로 수입할 수 있는 능력(수량)이 좋아졌다는 것을 의미한다. 연관검색어 : 순상품교역조건'),
 Document(metadata={'source': 'c:\\Users\\default.DESKTOP-T7K82N0\\moneymentor\\AI\\chatbot_economic_terms\\data\\한국은행_경제금융용어.txt'}, page_content='로렌츠곡선 1905년 미국의 통계학자인 로렌츠(M. Lorenz)가 소득의 불평등 정도를 측정하기 위해 제안한 것으로, 인구의 누적비율을 가로축에 소득의 누적점유율을 세로축에 놓고 이들의 관계를 그림으로 표시한 곡선을 말한다. 모든 사람의 소득이 일정하다면 인구가 누적되어도 소득 누적액이 일정할 것이므로 로렌츠 곡선(Lorenz Curve)은 아래 그림의 OO′선과 같은 대각선이 된다. 반면 소득이 불평등하다면 처음에는 소득이 적은 사람들의 누적액이 더해져 그래프의 기울기가 완만하다가 뒤로 갈수록 소득이 많은 사람들의 누적액이 더해지